In [120]:
import os
import sys

In [4]:
import json
import pandas as pd

In [3]:
schema_file_path = r'data\retail_db\schemas.json'

In [5]:
with open(schema_file_path, 'r') as schema_file:
    data_schema = json.load(schema_file)

In [27]:
def get_column_name(schema, category, sort_key='column_position'):
    try:
        if category in schema.keys():
            category_data = schema[category]
            columns = sorted(category_data, key=lambda col: col[sort_key])
            return [column['column_name'] for column in columns] 
        else:
            return None
    except Exception as ex:
        print(ex)

In [29]:
customer_columns = get_column_name(data_schema, 'customers')

In [30]:
order_columns = get_column_name(data_schema, 'orders')

In [31]:
customer_columns

['customer_id',
 'customer_fname',
 'customer_lname',
 'customer_email',
 'customer_password',
 'customer_street',
 'customer_city',
 'customer_state',
 'customer_zipcode']

In [63]:
order_columns

['order_id', 'order_date', 'order_customer_id', 'order_status']

In [62]:
orders_df = pd.read_csv(
    filepath_or_buffer=r'data\retail_db\orders\part-00000',
    names=order_columns
)

In [64]:
customers_df = pd.read_csv(
    filepath_or_buffer=r'data\retail_db\customers\part-00000',
    names=customer_columns
)

In [65]:
customers_df.head()

,customer_id,customer_fname,customer_lname,customer_email,customer_password,customer_street,customer_city,customer_state,customer_zipcode
0,1,Richard,Hernandez,XXXXXXXXX,XXXXXXXXX,6303 Heather Plaza,Brownsville,TX,78521
1,2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126
2,3,Ann,Smith,XXXXXXXXX,XXXXXXXXX,3422 Blue Pioneer Bend,Caguas,PR,725
3,4,Mary,Jones,XXXXXXXXX,XXXXXXXXX,8324 Little Common,San Marcos,CA,92069
4,5,Robert,Hudson,XXXXXXXXX,XXXXXXXXX,10 Crystal River Mall,Caguas,PR,725


In [67]:
orders_df.head()

,order_id,order_date,order_customer_id,order_status
0,1,2013-07-25 00:00:00.0,11599,CLOSED
1,2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT
2,3,2013-07-25 00:00:00.0,12111,COMPLETE
3,4,2013-07-25 00:00:00.0,8827,CLOSED
4,5,2013-07-25 00:00:00.0,11318,COMPLETE


## JOIN

In [69]:
customers_df.set_index('customer_id')


,customer_fname,customer_lname,customer_email,customer_password,customer_street,customer_city,customer_state,customer_zipcode
customer_id,,,,,,,,
1,Richard,Hernandez,XXXXXXXXX,XXXXXXXXX,6303 Heather Plaza,Brownsville,TX,78521
2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126
3,Ann,Smith,XXXXXXXXX,XXXXXXXXX,3422 Blue Pioneer Bend,Caguas,PR,725
4,Mary,Jones,XXXXXXXXX,XXXXXXXXX,8324 Little Common,San Marcos,CA,92069
5,Robert,Hudson,XXXXXXXXX,XXXXXXXXX,10 Crystal River Mall,Caguas,PR,725
...,...,...,...,...,...,...,...,...
12431,Mary,Rios,XXXXXXXXX,XXXXXXXXX,1221 Cinder Pines,Kaneohe,HI,96744
12432,Angela,Smith,XXXXXXXXX,XXXXXXXXX,1525 Jagged Barn Highlands,Caguas,PR,725
12433,Benjamin,Garcia,XXXXXXXXX,XXXXXXXXX,5459 Noble Brook Landing,Levittown,NY,11756


In [70]:
customers_df = customers_df.set_index('customer_id')

In [71]:
orders_df = orders_df.set_index('order_customer_id')

In [72]:
customer_orders = customers_df.join(orders_df, how='inner')

In [73]:
customer_orders.head()

,customer_fname,customer_lname,customer_email,customer_password,customer_street,customer_city,customer_state,customer_zipcode,order_id,order_date,order_status
customer_id,,,,,,,,,,,
1,Richard,Hernandez,XXXXXXXXX,XXXXXXXXX,6303 Heather Plaza,Brownsville,TX,78521,22945,2013-12-13 00:00:00.0,COMPLETE
2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126,15192,2013-10-29 00:00:00.0,PENDING_PAYMENT
2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126,33865,2014-02-18 00:00:00.0,COMPLETE
2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126,57963,2013-08-02 00:00:00.0,ON_HOLD
2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126,67863,2013-11-30 00:00:00.0,COMPLETE


In [75]:
customer_orders.shape

(68883, 11)

In [79]:
customer_orders['full_name'] = customer_orders.apply(lambda row_data: f'{row_data.customer_fname} {row_data.customer_lname}', axis=1)

In [80]:
customer_orders['full_name'].head()

customer_id
1    Richard Hernandez
2         Mary Barrett
2         Mary Barrett
2         Mary Barrett
2         Mary Barrett
Name: full_name, dtype: object

In [84]:
customer_orders.reset_index(inplace=True)

In [107]:
customer_orders.columns

Index(['customer_id', 'customer_fname', 'customer_lname', 'customer_email',
       'customer_password', 'customer_street', 'customer_city',
       'customer_state', 'customer_zipcode', 'order_id', 'order_date',
       'order_status', 'full_name'],
      dtype='object')

In [85]:
customer_orders.head()

,customer_id,customer_fname,customer_lname,customer_email,customer_password,customer_street,customer_city,customer_state,customer_zipcode,order_id,order_date,order_status,full_name
0,1,Richard,Hernandez,XXXXXXXXX,XXXXXXXXX,6303 Heather Plaza,Brownsville,TX,78521,22945,2013-12-13 00:00:00.0,COMPLETE,Richard Hernandez
1,2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126,15192,2013-10-29 00:00:00.0,PENDING_PAYMENT,Mary Barrett
2,2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126,33865,2014-02-18 00:00:00.0,COMPLETE,Mary Barrett
3,2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126,57963,2013-08-02 00:00:00.0,ON_HOLD,Mary Barrett
4,2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126,67863,2013-11-30 00:00:00.0,COMPLETE,Mary Barrett


In [92]:
customer_orders.groupby(['customer_city', 'customer_id'])['customer_city'].agg(order_count='count')

order_count
customer_city customer_id             
Aguadilla     2261                   3
              6497                   3
              6627                   2
              7257                   3
              8460                   3
...                                ...
Zanesville    6740                   9
              6817                   4
              8862                   2
              10105                  7
              12180                  7

[12405 rows x 1 columns]

In [117]:
customer_orders[['order_id', 'order_date', 'full_name']].\
    sort_values(['order_id', 'order_date'], ascending=True)

,order_id,order_date,full_name
64227,1,2013-07-25 00:00:00.0,Mary Malone
1385,2,2013-07-25 00:00:00.0,David Rodriguez
67088,3,2013-07-25 00:00:00.0,Amber Franco
48868,4,2013-07-25 00:00:00.0,Brian Wilson
62682,5,2013-07-25 00:00:00.0,Mary Henry
...,...,...,...
4290,68879,2014-07-09 00:00:00.0,John Buckley
6130,68880,2014-07-13 00:00:00.0,Arthur Maddox
13909,68881,2014-07-19 00:00:00.0,Mary Smith
55366,68882,2014-07-22 00:00:00.0,Mary Mcfarland


In [122]:
os.makedirs('data/structured_zone', exist_ok=True)

In [141]:
result = customer_orders.groupby(['customer_state', 'customer_city']).agg(order_count=('customer_id', 'count')).reset_index()
result = result[['customer_state', 'customer_city', 'order_count']]


In [145]:
result = result.sort_values(['order_count'], ascending=False)

In [146]:
result

,customer_state,customer_city,order_count
483,PR,Caguas,25487
232,IL,Chicago,1488
392,NY,Brooklyn,1302
83,CA,Los Angeles,1285
410,NY,New York,676
...,...,...,...
94,CA,National City,10
582,WA,Sumner,10
360,NJ,Freehold,10
280,MA,Malden,9


In [147]:
result.to_csv('data/structured_zone/result.csv', index=False)

In [149]:
result.to_json('data/structured_zone/result.json',orient='records', lines=True)